In [1]:
#Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv
/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload


In [2]:
monthly_riders_since_2008 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Monthly_Ridership___Traffic_Data__Beginning_January_2008.csv')

#No missing values
monthly_riders_since_2008.dropna(axis=1, inplace=True)

#Filter to only include Subway and no other MTA agency
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Agency'] == 'Subway'] 

#Rename "Month" column into Date
monthly_riders_since_2008.rename(columns = {'Month':'Date'}, inplace = True)

#Polish Date column and conver to datetime
monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].str.replace('/01/', '/', regex=True)
monthly_riders_since_2008['Date'] = pd.to_datetime(monthly_riders_since_2008['Date'], format='%m/%Y')
#monthly_riders_since_2008['Date'] = monthly_riders_since_2008['Date'].dt.strftime('%m/%Y')

#Break out date into coumns for date, month, and year
monthly_riders_since_2008['Month'] = monthly_riders_since_2008['Date'].dt.month
monthly_riders_since_2008['Year'] = monthly_riders_since_2008['Date'].dt.year

#Drop Date column now
monthly_riders_since_2008.drop(['Agency'], inplace=True, axis=1)

#Filter out the year 2024
monthly_riders_since_2008 = monthly_riders_since_2008[monthly_riders_since_2008['Year'] < 2024] 

#Groupby Year and look at mean ridership
#monthly_riders_since_2008['Mean Ridership By Year'] = monthly_riders_since_2008.groupby('Year')['Ridership'].transform(lambda x: x.mean())
#monthly_riders_since_2008['Total Ridership By Year'] = monthly_riders_since_2008.groupby('Year')['Ridership'].transform(lambda x: x.sum())

#Rearrange columns (Remeber to Add above columns if you want to show Mean and Total Amounts)
monthly_riders_since_2008 = monthly_riders_since_2008.reindex(['Date', 'Month', 'Year', 'Ridership'], axis=1)


#groued_by_year = monthly_riders_since_2008[['Year', 'Mean Ridership By Year', 'Total Ridership By Year']].drop_duplicates()
#groued_by_year

#sns.barplot(data=by_year, x='Year', y='Mean Ridership By Year')
#plt.show()



monthly_riders_since_2008

,Date,Month,Year,Ridership
238,2018-01-01,1,2018,134683435
242,2018-02-01,2,2018,127432835
246,2018-03-01,3,2018,143982923
251,2018-04-01,4,2018,141950369
255,2018-05-01,5,2018,150320833
...,...,...,...,...
673,2023-08-01,8,2023,94141961
679,2023-09-01,9,2023,95253595
686,2023-10-01,10,2023,103949018
693,2023-11-01,11,2023,98511566


In [3]:
daily_riders_since_2020 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Daily_Ridership_Data__Beginning_2020.csv', parse_dates=['Date'])

#No missing values
daily_riders_since_2020.dropna(axis=1, inplace=True)

#Drop unccesary columns that belong to other agencies outside of the subway 
daily_riders_since_2020.drop(daily_riders_since_2020.iloc[:, 3:], inplace=True, axis=1)

#Convert to Datetime
#daily_riders_since_2020['Date'] = pd.to_datetime(daily_riders_since_2020['Date'])

#Break out date into columns for date, month, and year
daily_riders_since_2020['Day'] = daily_riders_since_2020['Date'].dt.day
daily_riders_since_2020['Month'] = daily_riders_since_2020['Date'].dt.month
daily_riders_since_2020['Year'] = daily_riders_since_2020['Date'].dt.year
daily_riders_since_2020['Day of Week'] = daily_riders_since_2020['Date'].dt.dayofweek

#Filter out the year 2024
daily_riders_since_2020 = daily_riders_since_2020[daily_riders_since_2020['Year'] < 2024] 

#Sort by date
daily_riders_since_2020.sort_values('Date', ascending=True, inplace=True)

#Renaming some columns
daily_riders_since_2020.rename(columns={'Subways: Total Estimated Ridership':'Total Estimated Ridership', 
                                        'Subways: % of Comparable Pre-Pandemic Day': '% Comparable to Pre-Pandemic Days'}, 
                               inplace=True)

#Rearrange columns (Remeber to Add above columns if you want to show Mean and Total Amounts)
daily_riders_since_2020 = daily_riders_since_2020.reindex(['Date', 'Day', 'Month', 'Year', 'Day of Week', 'Total Estimated Ridership', '% Comparable to Pre-Pandemic Days'], axis=1)


daily_riders_since_2020

,Date,Day,Month,Year,Day of Week,Total Estimated Ridership,% Comparable to Pre-Pandemic Days
1543,2020-03-01,1,3,2020,6,2212965,0.97
1542,2020-03-02,2,3,2020,0,5329915,0.96
1541,2020-03-03,3,3,2020,1,5481103,0.98
1540,2020-03-04,4,3,2020,2,5498809,0.99
1539,2020-03-05,5,3,2020,3,5496453,0.99
...,...,...,...,...,...,...,...
148,2023-12-27,27,12,2023,2,2912007,0.55
147,2023-12-28,28,12,2023,3,3064841,0.57
146,2023-12-29,29,12,2023,4,3198885,0.60
145,2023-12-30,30,12,2023,5,2440211,0.74


In [4]:
#df3 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', iterator=True, chunksize=1000)
hourly_ridership_since_2022 = pd.read_csv('/kaggle/input/mta-daily-ridership-data-beginning-2020/MTA_Subway_Hourly_Ridership__Beginning_February_2022.csv.crdownload', low_memory=False)


#dropping missing values
hourly_ridership_since_2022.dropna(inplace=True)

#Convert timestamp column to Datetime
hourly_ridership_since_2022['transit_timestamp'] = pd.to_datetime(hourly_ridership_since_2022['transit_timestamp'], format='%m/%d/%Y %I:%M:%S %p')

#Breakout timestamp info into diffeent columns
hourly_ridership_since_2022['Day'] = hourly_ridership_since_2022['transit_timestamp'].dt.day
hourly_ridership_since_2022['Month'] = hourly_ridership_since_2022['transit_timestamp'].dt.month
hourly_ridership_since_2022['Year'] = hourly_ridership_since_2022['transit_timestamp'].dt.year
hourly_ridership_since_2022['Day of Week'] = hourly_ridership_since_2022['transit_timestamp'].dt.dayofweek

#query to further filter
hourly_ridership_since_2022 = hourly_ridership_since_2022.query('borough != "Staten Island" and transit_mode != "tram" and station_complex_id != "TRAM1"')

#Convert Station ID to int (theres a station id called 'TRAM1'? check if its true)
hourly_ridership_since_2022['station_complex_id'] = hourly_ridership_since_2022['station_complex_id'].astype(int)

#Create new LagLog column with the latitude and longtitude combine
hourly_ridership_since_2022['lat_long'] = hourly_ridership_since_2022['latitude'].astype(str) + ', ' + hourly_ridership_since_2022['longitude'].astype(str)

#Drop unccesary columns for our analysis
hourly_ridership_since_2022.drop(columns=['payment_method', 'fare_class_category', 'transfers', 'Georeference', 'transit_mode'], inplace=True, axis=1)    

#Rearrange columns (Remeber to Add above columns if you want to show Mean and Total Amounts)
hourly_ridership_since_2022 = hourly_ridership_since_2022.reindex(['transit_timestamp', 'Month', 'Day', 'Year', 'Day of Week', 'station_complex_id', 'station_complex', 
                                                                 'borough', 'ridership', 'latitude', 'longitude', 'lat_long'], axis=1)
    
    
hourly_ridership_since_2022

,transit_timestamp,Month,Day,Year,Day of Week,station_complex_id,station_complex,borough,ridership,latitude,longitude,lat_long
0,2023-02-12 18:00:00,2,12,2023,6,164,"34 St-Penn Station (A,C,E)",Manhattan,84.0,40.752289,-73.993393,"40.752288818359375, -73.99339294433594"
1,2023-03-28 14:00:00,3,28,2023,1,164,"34 St-Penn Station (A,C,E)",Manhattan,123.0,40.752289,-73.993393,"40.752288818359375, -73.99339294433594"
2,2023-01-11 02:00:00,1,11,2023,2,164,"34 St-Penn Station (A,C,E)",Manhattan,3.0,40.752289,-73.993393,"40.752288818359375, -73.99339294433594"
3,2023-06-08 17:00:00,6,8,2023,3,164,"34 St-Penn Station (A,C,E)",Manhattan,200.0,40.752289,-73.993393,"40.752288818359375, -73.99339294433594"
4,2023-07-25 16:00:00,7,25,2023,1,164,"34 St-Penn Station (A,C,E)",Manhattan,872.0,40.752289,-73.993393,"40.752288818359375, -73.99339294433594"
...,...,...,...,...,...,...,...,...,...,...,...,...
45109797,2023-08-19 06:00:00,8,19,2023,5,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,1.0,40.749718,-73.987823,"40.74971771240234, -73.98782348632812"
45109798,2023-10-06 10:00:00,10,6,2023,4,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,5.0,40.749718,-73.987823,"40.74971771240234, -73.98782348632812"
45109799,2023-06-01 16:00:00,6,1,2023,3,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,11.0,40.749718,-73.987823,"40.74971771240234, -73.98782348632812"
45109800,2023-05-28 14:00:00,5,28,2023,6,607,"34 St-Herald Sq (B,D,F,M,N,Q,R,W)",Manhattan,1662.0,40.749718,-73.987823,"40.74971771240234, -73.98782348632812"


In [5]:
monthly_riders_since_2008.to_csv('/kaggle/working/monthly_riders_since_2008.csv', index=False)

daily_riders_since_2020.to_csv('/kaggle/working/daily_riders_since_2020.csv', index=False)

hourly_ridership_since_2022.to_csv('/kaggle/working/hourly_ridership_since_2022.csv', index=False)